## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.

## 목적:
- Sentence Transformers를 사용하여 텍스트 데이터를 벡터로 변환
- 변환된 벡터를 Qdrant에 저장하여 검색 가능하게 만듦

## 왜 필요한가?
- 자연어 데이터를 벡터로 변환하면 고유한 의미를 인식하고 검색 가능해짐
- AI 모델이 유사한 문장을 효과적으로 찾을 수 있도록 도와줌

## 주요 개념:
- **Sentence Transformers**: 텍스트를 의미론적 벡터로 변환하는 강력한 모델
- **임베딩(Embedding)**: 텍스트, 이미지 등의 데이터를 고차원 벡터로 변환하는 과정
- **유사도 검색(Similarity Search)**: 벡터 간 거리를 계산하여 유사한 항목을 찾는 기법

In [7]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams

In [3]:
# Qdrant 클라이언트 연결 (아래에서 오류가 발생한다면 qdrant 서버를 띄워주세요)
qdrant_client = QdrantClient(host="localhost", port=6333)

In [4]:
# 컬렉션 존재 여부 확인 후 생성 
collection_name = "ex2_sample_vectors"

if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
    )

In [5]:
# Sentence Transformers 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
# 여러 개의 텍스트 데이터 준비
text_data = [
    "Vector databases are powerful!",
    "Qdrant is a fast and efficient vector search engine.",
    "Sentence Transformers generate embeddings from text.",
    "Search optimization using vector-based similarity."
]

# 각각의 텍스트를 벡터로 변환 후 Qdrant에 저장
for i, text in enumerate(text_data):
    embedding_vector = embeddings.embed_query(text)  # 변경된 임베딩 메서드 사용
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[
            models.PointStruct(id=i, vector=embedding_vector, payload={"text": text})
        ]
    )

print("모든 텍스트 데이터를 Qdrant에 저장 완료!")

모든 텍스트 데이터를 Qdrant에 저장 완료!
